# **Imports**

In [235]:
import pandas as pd
from dataclasses import dataclass
from openpyxl import load_workbook
from datetime import datetime, timedelta
import numpy as np
pd.set_option('display.max_columns', None)

# **Constants**

In [236]:
usefull_events_for_period: str = '../main_datasets/processed_datasets/События_за_период_01.01.2024-30.04.2024____pivot_table.csv'

@dataclass(frozen=True)
class Pathes:
    # moec_connection_scheme: str = '../main_datasets/processed_datasets/7. Схема подключений МОЭК.csv'
    events_for_2023: str = '../main_datasets/processed_datasets/События за период_01.10.2023-31.12.2023.xlsx____uploading_table.csv'
    events_for_2024: str = '../main_datasets/processed_datasets/События_за_период_01.01.2024-30.04.2024____uploading_table.csv'
    turn_off_water_2023: str = '../main_datasets/processed_datasets/6. Плановые-Внеплановые отключения 01.10.2023-30.04.2023.csv'



In [237]:
columns_norm_forms = {
    'УНОМ': 'UNOM',
    'Учётный номер объекта адресации в БД БТИ (кроме помещения и комнаты)': 'UNOM'
}

In [238]:
usefull_columns = [
    'Дата', 'Месяц/Год', 'Ошибки', 'Объём поданого теплоносителя в систему ЦО', 
    'Объём обратного теплоносителя из системы ЦО', 'Разница между подачей и обраткой(Подмес)',
    'Разница между подачей и обраткой(Утечка)', 'Температура подачи', 'Температура обратки',
    'Наработка часов счётчика', 'Расход тепловой энергии', 'Ошибки'
]

In [239]:
usefull_events_dataset = pd.read_csv(usefull_events_for_period)
usefull_events = usefull_events_dataset['Названия строк']
usefull_events

0                                               P1 <= 0
1                                               P2 <= 0
2                                              T1 < min
3                                              T1 > max
4                    Аварийная протечка труб в подъезде
5                                        Крупные пожары
6                           Отсутствие отопления в доме
7                              Протечка труб в подъезде
8                      Сильная течь в системе отопления
9               Температура в квартире ниже нормативной
10    Температура в помещении общего пользования ниж...
11                             Течь в системе отопления
12                                           Общий итог
Name: Названия строк, dtype: object

# **Functions**

In [232]:
def make_timeline_for_year(year: int=2023):
    data_1 = np.arange(datetime(year=year, month=1, day=1), datetime(year=year, month=5, day=1), timedelta(hours=1)).tolist()
    data_2 = np.arange(datetime(year=year, month=10, day=1), datetime(year=year+1, month=1, day=1), timedelta(hours=1)).tolist()
    return np.array(data_1+data_2)


def make_timeline_for_period(year: int=2022):
    s = datetime(year=year, month=10, day=1)
    e = datetime(year=year+1, month=5, day=1)
    data = np.arange(s, e, timedelta(minutes=1)).tolist()
    return data

In [176]:
def column_preprocessor(columns: pd.Index):
    return pd.Index([columns_norm_forms[i] if i in columns_norm_forms else i for i in columns])


def replace_bad_spaces(df: pd.Index) -> pd.DataFrame:
    c = df.map(lambda a: str(a).rstrip().lstrip())
    return c


def remove_useless_columns(df: pd.DataFrame):
    df = df.loc[:, df.columns.isin(usefull_columns)]
    return df


def get_only_valid_rows_by_event(df: pd.DataFrame):
    if 'Наименование' in df.columns:
        return df.loc[df['Наименование'].isin(usefull_events)]
    return df


In [177]:
def data_frame_loader(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = column_preprocessor(df.columns)
    df.columns = replace_bad_spaces(df.columns)
    df = get_only_valid_rows_by_event(df)
    return df

# **Show of tables**

In [178]:
pd.read_csv(Pathes.events_for_2024).head()

,Наименование,Источник,Дата создания во внешней системе,Дата закрытия,Округ,УНОМ,Адрес,Дата и время завершения события во внешней системе
0,Лифт требует ремонта,EDC,2024-03-10 14:52:57.000000,2024-03-10 14:56:08.183000,ВАО,64137.0,внутригородская территория муниципальный округ...,2024-03-10 16:35:37.366000
1,Лифт требует ремонта,EDC,2024-01-14 16:57:54.000000,2024-01-14 17:03:17.961000,ВАО,380.0,внутригородская территория муниципальный округ...,2024-01-15 10:09:27.502000
2,Лифт требует ремонта,EDC,2024-01-14 14:00:01.000000,2024-01-14 14:04:48.621000,ВАО,380.0,внутригородская территория муниципальный округ...,2024-01-14 16:59:01.571000
3,Протечка с кровли,EDC,2024-01-24 12:06:28.000000,2024-01-24 12:09:12.944000,ВАО,380.0,внутригородская территория муниципальный округ...,2024-01-24 14:05:31.400000
4,Нет связи с УСПД,ASUPR,2024-03-11 22:12:07.000000,2024-03-12 10:40:58.780000,ВАО,380.0,внутригородская территория муниципальный округ...,2024-03-12 10:40:58.991000


# **Loading dataset** 

In [184]:
dfs = pd.concat([data_frame_loader(Pathes.__getattribute__(Pathes, i)) for i in Pathes.__annotations__.keys()], axis=0)
dfs.index = pd.to_datetime(dfs['Дата создания во внешней системе'])

In [181]:
unom_ids = np.unique(sum([i['UNOM'].to_list() for i in dfs], []))
len(unom_ids)

4357

In [219]:
from tqdm import tqdm

In [234]:
year = [2022, 2023, 2024]
for uid in tqdm(unom_ids):
    for y in year:
        anomaly_df = pd.DataFrame(columns=dfs.columns, index=make_timeline_for_period(y))
        df = dfs.loc[dfs['UNOM'] == uid]
        df.index = df.index.round(freq='min')
        indexes = df.index.intersection(anomaly_df.index)
        anomaly_df.loc[indexes] = df.loc[indexes]
        anomaly_df.to_csv(f'../main_datasets/builded_datasets/anomaly_timelines/period_{y}/{int(uid)}_{y}.csv')


    

  0%|          | 6/4357 [00:47<9:28:06,  7.83s/it]


ValueError: cannot reindex on an axis with duplicate labels